<a href="https://colab.research.google.com/github/angelovelloso/find-public-expenses/blob/main/Alura_Desafio_Imers%C3%A3o_Projeto_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto: Identificar, sumarizar e classificar futuras despesas públicas

## 1. Objetivo


Como servidor do Governo do Estado de Rondônia e trabalhando diretamente na extração e organização dos dados relacionados às despesas, conseguimos ter uma visão bastante profunda e detalhada sobre os gastos públicos comprometidos ou incorridos utilizando tecnologias e metodologias de Engenharia de Dados.

Isso se dá porque há uma adesão total ao Sistema Financeiro adotado na gestão do ciclo orçamentário, que envolve a definição dos limites de crédito orçamentário e passa pelo Empenho, Liquidação e Pagamento.

O principal gargalo está em identificar ou gerenciar o ciclo de licitação e contratação, uma vez que esses processos acontecem em diversas unidades descentralizadamente e não são gerenciadas ou imputadas em nenhum sistema de informação que dê suporte ao processo e sirva de fonte da informação.

Mapear iniciativas de aquisições públicas antes que se tornem compromissos na fase de Empenho interessa à administração especialmente em um contexto de frustração de receitas e necessidade de limitação de empenho e movimentação financeira.

Este projeto olha para essa necessidade e surge a partir do desafio promovido pela Alura na Imersão Inteligência Artificial 2ª Edição e propõe o uso da poderosa tecnologia da API do **Gemini**, a IA Generativa do Google.

Neste contexto **três objetivos** serão perseguidos:
* Identificar e sistematizar as novas iniciativas de aquisição pública;
* Sumarizar o que está a Administração Pública pretende adquirir;
* Classificar de forma útil as futuras despesas.

## 2. Abordagem

Para atingir os objetivos delineados, a abordagem proposta passa por:
1. Monitorar a publicação do Diário Oficial do Estado de Rondônia;
2. A cada nova edição, utilizar a API do **Gemini** para analisar o documento e **identificar** novas iniciativas de aquisição pública que estejam sendo divulgadas, **sistematizando** e **estruturando** a informação de forma útil;
3. Considerando que estes documentos (Editais, Atas, Termos de Referência) apresentam um grande volume de informação e detalhes, a API do **Gemini** será responsável por também **sumarizar** os documentos e **extrair** dados úteis para descrever de forma objetivo o que a Administração Pública pretende adquirir;
4. Ao final, a partir dos dados sumarizados obtidos e, visando direcionar um suposto revisor de despesas, a expectativa é que cada objeto seja **classificado** de forma pré-estabelecida e para o qual serão adotadas como referência as categorias previstas no § 1° do Art. 57 da Lei de Diretrizes Orçamentária do Estado de Rondônia para o exercício de 2024.
5. Essas categorias dizem respeito às despesas que devem ser observadas prioritariamente no processo de limitação de despesas.

Como referência, segue o texto integral do artigo e as categorias enumeradas:

> `Art. 57. Se verificado, ao final de um bimestre, que a realização da receita poderá não comportar o cumprimento das metas de resultado primário ou nominal estabelecidas no Anexo de Metas Fiscais, na forma do artigo 9° da Lei Complementar Federal n° 101, de 2000, os Poderes, o Ministério Público, a Defensoria Pública do Estado e o Tribunal de Contas do Estado promoverão, por ato próprio e nos montantes necessários, nos 30 (trinta) dias subsequentes, limitação de empenho e movimentação financeira, de forma proporcional à queda de arrecadação estimada nas fontes de recursos específicas que`
> `suportam as dotações orçamentárias do respectivo Poder ou Órgão.`
>
> `§ 1° O Poder Executivo de forma proporcional às suas dotações adotará medidas necessárias para o cumprimento do caput, observadas as respectivas fontes de recursos, em especial, nas seguintes despesas:`
>
> `I - contrapartida para projetos ou atividades vinculadas a recursos oriundos de fontes extraordinárias, como transferências voluntárias, operações de crédito e alienação de ativos, desde que ainda não comprometidos;`
> `II - obras em geral, cuja fase ou etapa ainda não esteja iniciada;`
> `III - aquisição de combustíveis e derivados, destinada à frota de veículos, exceto dos setores de saúde, educação e segurança pública;`
> `IV - dotação para material de consumo e outros serviços de terceiros para as diversas atividades;`
> `V - diárias de viagem;`
> `VI - festividades, homenagens, recepções e demais eventos da mesma natureza;`
> `VII - despesas com publicidade institucional; e`
> `VIII - horas-extras.`

## 3. Preparação do Ambiente

In [1]:
# Instaling Generativa AI Library

!pip install -U -q google-generativeai PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.5 MB/s eta 0:00:00


In [2]:
# Import libraries

from google.colab import userdata
from pathlib import Path

import duckdb
import google.generativeai as genai
import hashlib
import json
import os
import PyPDF2
import requests

In [3]:
# Define utility functions

def baixar_pdf(url):
  response = requests.get(url)
  path = Path(url)
  new_path = 'data/input/' + path.name
  with open(new_path, 'wb') as f:
    f.write(response.content)

  return f.name

def extract_pdf_pages(pathname: str) -> list[str]:
    parts = [f"--- INÍCIO DO PDF ${pathname} ---"]

    with open(pathname, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)

        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()
            parts.append(f"--- PÁGINA {page_num+1} --- \n {text}")

    return parts

In [4]:
# Load and set API Key

genai.configure(api_key=userdata.get('GEMINI_API_KEY'))

In [5]:
# Create directories
os.mkdir('data')
os.mkdir('data/input')
os.mkdir('data/output')
os.mkdir('data/db')

## 4. Donwload e extração do texto do PDF

In [6]:
# Download file

arquivo = baixar_pdf(
    url='https://diof.ro.gov.br/data/uploads/2024/05/DOE-08.05.2024.pdf'
)

In [7]:
# Extract text to variable

pdf_content = extract_pdf_pages(arquivo)

## 5. Definição dos parâmetros do Modelo

In [8]:
# Set up the model

generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

prompt_template = '''
    Aja como um revisor responsável por identificar e rever despesas públicas.

    Instruções:
    1. A partir do Diário Oficial do Estado de Rondônia, identifique as publicações que representam a iniciativa de o poder público fazer novas aquisições de bens, produtos ou serviços.
    2. As iniciativas podem ser licitações, pregões, atas, termos de referência ou qualquer espécie de documento que torne público a intenção de a administração pública adquirir bens, produtos ou serviços.
    3. Não liste Termos de Homologação referentes à prestação de contas de atendimento de diárias.
    4. Para cada publicação identificada, apresente as seguintes informações:
        - Página onde a publicação foi identificada;
        - Documento ou identificação da publicação;
        - Unidade do governo que é responsável pela aquisição;
        - Valor total da despesa, se houver;
	    - Fornecedor identificado pelo CNPJ e nome ou razão social, se houver;
        - Em uma única frase, uma descrição objetiva do que a Administração Pública pretende adquirir;
        - Em uma lista, uma descrição curta de cada item que está sendo adquirido, a quantidade e o valor esperado.
    5. Apresente as informações em formato JSON para posterior tratamento dos dados gerados, usando as seguintes chaves:
        - Página;
        - Documento;
        - Unidade;
        - Valor_Total;
        - Fornecedor;
        - Descrição;
        - Itens.
  '''

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

## 6. Execução das solicitações a API do Gemini

Devido ao tamanho do documento, ao número de tokens e especialmente a possibilidade de time out da requisiçao, serão feitas requisições a cada 100 páginas.

In [9]:
# Create specific outputdir

path = Path(arquivo)
output_dir = f'data/output/{path.name.replace(".pdf","")}'
os.mkdir(output_dir)

In [10]:
# Execute API to extract structured data

chunk_size = 10
total_pages = len(pdf_content)
chunk_blocks = int(total_pages / chunk_size)

if total_pages % chunk_size != 0:
  chunk_blocks += 1

for block in range(chunk_blocks):

    start_page = 1 if block == 0 else (block * chunk_size) - 1 # Add one pages before
    end_page = ((block + 1) * chunk_size) + 1 # Add one pages after

    if end_page > total_pages:
        end_page = total_pages

    prompt_parts = [
        *pdf_content[0:1],
        *pdf_content[start_page:end_page],
        prompt_template,
    ]

    print(f'Chamando API para bloco {block}, página {start_page} até {end_page} ...')

    response = model.generate_content(
        prompt_parts,
        request_options={'timeout': 300})

    try:
        json_block = response.text.replace('```', '').replace('json', '')
        dados_aquisicoes = json.loads(json_block)

        arquivo = output_dir + f'/aquisicoes_block_{block}.json'

        with open(arquivo, 'w') as f:
            json.dump(dados_aquisicoes, f, indent=4)

    except ValueError:
        print(f'Bloco {block} sem aquisições.')
        continue

Chamando API para bloco 0, página 1 até 11 ...
Chamando API para bloco 1, página 9 até 21 ...
Chamando API para bloco 2, página 19 até 31 ...
Chamando API para bloco 3, página 29 até 41 ...
Chamando API para bloco 4, página 39 até 51 ...
Bloco 4 sem aquisições.
Chamando API para bloco 5, página 49 até 61 ...
Chamando API para bloco 6, página 59 até 71 ...
Chamando API para bloco 7, página 69 até 81 ...
Chamando API para bloco 8, página 79 até 91 ...
Chamando API para bloco 9, página 89 até 101 ...
Chamando API para bloco 10, página 99 até 111 ...
Chamando API para bloco 11, página 109 até 121 ...
Chamando API para bloco 12, página 119 até 131 ...
Chamando API para bloco 13, página 129 até 141 ...
Chamando API para bloco 14, página 139 até 151 ...
Chamando API para bloco 15, página 149 até 161 ...
Chamando API para bloco 16, página 159 até 171 ...
Chamando API para bloco 17, página 169 até 181 ...


TooManyRequests: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).

## 7. Criação e estruturação do banco de dados

In [19]:
con = duckdb.connect('data/db/database.duckdb')

con.sql(
    '''
    create or replace table aquisicoes (
        pagina varchar,
        documento varchar,
        unidade varchar,
        valor_total_str varchar,
        fornecedor varchar,
        descricao varchar,
        arquivo varchar
    )
    '''
)

In [20]:
con.sql('describe aquisicoes')

┌─────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│   column_name   │ column_type │  null   │   key   │ default │  extra  │
│     varchar     │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ pagina          │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ documento       │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ unidade         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ valor_total_str │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ fornecedor      │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ descricao       │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ arquivo         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

In [21]:
for json_file in os.listdir(output_dir):

    print(f'Lendo {json_file} ...')

    con.sql(f'''
        insert into aquisicoes
        select
            Página,
            Documento,
            Unidade,
            Valor_total,
            Fornecedor,
            Descrição,
            '{json_file}'
        from read_json('{output_dir}/{json_file}') f
    ''')

Lendo aquisicoes_block_10.json ...
Lendo aquisicoes_block_0.json ...
Lendo aquisicoes_block_2.json ...
Lendo aquisicoes_block_15.json ...
Lendo aquisicoes_block_13.json ...
Lendo aquisicoes_block_11.json ...
Lendo aquisicoes_block_7.json ...
Lendo aquisicoes_block_6.json ...
Lendo aquisicoes_block_1.json ...
Lendo aquisicoes_block_8.json ...
Lendo aquisicoes_block_16.json ...
Lendo aquisicoes_block_9.json ...
Lendo aquisicoes_block_3.json ...
Lendo aquisicoes_block_12.json ...
Lendo aquisicoes_block_14.json ...
Lendo aquisicoes_block_5.json ...


In [27]:
con.sql(f"""
        select
            pagina,
            documento,
            descricao
        from aquisicoes
""")

┌─────────┬──────────────────────┬─────────────────────────────────────────────────────────────────────────────────────┐
│ pagina  │      documento       │                                      descricao                                      │
│ varchar │       varchar        │                                       varchar                                       │
├─────────┼──────────────────────┼─────────────────────────────────────────────────────────────────────────────────────┤
│ 99-101  │ ATA DE REGISTRO DE…  │ Registro de preços para futura e eventual aquisição de TABLET AVANÇADO.             │
│ 103-110 │ ATA DE REGISTRO DE…  │ Registro de preço para futura e eventual aquisição de materiais de consumo (exped…  │
│ 2       │ LEI COMPLEMENTAR N…  │ Criação de novos cargos no quadro administrativo do Ministério Público do Estado …  │
│ 28      │ EXTRATO 1-EXTRATO:…  │ Aquisição de serviços de segurança para a Secretaria de Estado da Saúde.            │
│ 29      │ EXTRATO 1-EXTRATO:… 